// コメント：最悪実行時間 O(n!) / n:名詞数

## 1. データの正規化

In [ ]:
!pip install janome
from janome.tokenizer import Tokenizer
!pip install jaconv
import jaconv

In [ ]:
data = ("""
元日の夜。
エヌ氏がうつらうつらしていると、だれかが訪れてくるけはいがした。
「どなたですか」
と身をおこすと、大黒さまだった。
どことなく神々しく、じつに福々しい顔をしている。
また、肩には大きな袋をしょっている。
しかし、念のために聞いてみることにした。
「もしかしたら、大黒さまではございませんか」
「そうだ。おまえに福を授けにきたのだ」
　本物にまちがいないようだ。
エヌ氏は大声をあげた。
「ほんとですか」
「もちろんだ。わたしの力で幸福になった者は、数えきれない」
「しめた。ばんざい。では、すぐに……」
　エヌ氏は飛びあがり、手を出した。
そのようすを見ながら、大黒は言った。
「どうやら、おまえはあわて者の性格のようだな。それを取り除けば……」
「これがあわてずにいられますか。さて、なにをいただくとするか。このチャンスをのがさず……」
　頭のなかで、エヌ氏は巨額な数字を並べはじめた。
「おまえは欲ばりなところがあるな。それを取り除けば……」
「なんと言われようが、かまうものですか。そうだ。その袋のなかのものをみんな下さい」
　エヌ氏は目を輝かせて指さした。
大黒は困ったような顔をした。
「おまえには変なものに目をつける性格があるな。それを取り除けば……」
「さっきから、取り除くことぼかり言っている。そんなことは、精神分析医にでも任せておけばいい。さあ、下さい」
「これだけはやれない。どうも、おまえには強引で無茶なところがあるぞ」
「それを取り除けばというんでしょう。いらいらしてきた。こうなったら、腕ずくでもちょうだいする」
「まて、やめろ」
　大黒はさえぎったが、エヌ氏はすばやく飛びつき、奪ってしまった。
そして、袋の口を開き、のぞきこんだ。
なんともいえぬ、いやな気分がたちのぼってきた。
エヌ氏は顔をしかめながら聞いた。
「なんです、これは」
「そもそも福とは、その障害になっているものを取り除けば、簡単に手にはいる。すなわち福を与える結果になり、それがわたしの役目だ。ほうぼうで集めてきたのが袋にいっぱいはいっていた。それをあけて吸い込むとは、おまえはなんという……」""")

In [ ]:
t = Tokenizer()
for token in t.tokenize(data):
  print(token)

In [ ]:
n_lists = []

for token in t.tokenize(data):
  if token.part_of_speech.startswith("名詞"):
    if not (token.reading == "*"):
      n_lists.append((token.reading,token.surface))
    else:
      n_lists.append((jaconv.hira2kata(token.surface),token.surface))
print(n_lists)

In [ ]:
def remove_duplicates(lst):
    return list(set(lst))

In [ ]:
nr_list = remove_duplicates(n_lists)
print(nr_list)

In [ ]:
nrs_list = sorted(nr_list)
print(nrs_list)

## 2. アルゴリズム

In [ ]:
### 方針
# set の中身を (reading, surface, first_word, end_word) の構成にする
# 各単語について, end_word → first_word を検索し, 各単語について最長のしりとりリストを保管
# 各単語のしりとりリストを結合し, 最長のしりとりリストを構築

### 2.1 正規化 (setの再構築)

In [ ]:
def process_tuples(tuple_list):
    result = []
    for t in tuple_list:
        first_elem = t[0]
        first_char = first_elem[0]
        last_char = first_elem[-1]
        new_t = t + (first_char, last_char)
        result.append(new_t)
    return result

In [ ]:
word_list = process_tuples(nrs_list)
print(word_list)

### 2.2 各単語の最長しりとりリスト

In [ ]:
def build_graph(words):
  # 各単語ごとに連結可能な単語リスト辞書を作成
    graph = {}
    for w1 in words:
        graph[w1] = []
        for w2 in words:
            if w1 != w2 and w1[3] == w2[2]:
                graph[w1].append(w2)
    return graph

In [ ]:
word_link_dict = build_graph(word_list)
print(word_link_dict)

In [ ]:
def dfs(word_tuple, tuple_list, visited):
  #再起を用いて全探索 (最悪実行時間 O(n!))
    visited.add(word_tuple)
    longest_path = [word_tuple]
    for next_tuple in tuple_list:
        if next_tuple not in visited:
            if word_tuple[3] == next_tuple[2]:  # 末尾文字 == 次の先頭文字
                path = dfs(next_tuple, tuple_list, visited.copy())
                if len(path) + 1 > len(longest_path):
                    longest_path = [word_tuple] + path
    return longest_path

In [ ]:
#使用例
long_path = dfs(word_list[0],word_list,set())
print(long_path)

In [ ]:
def longest_shiritori(tuple_list):
  #全ての単語に実行して各単語について最長のしりとりリストを保管
    results = {}
    for t in tuple_list:
        path = dfs(t, tuple_list, set())
        words_only = [p[1] for p in path]  # 元の単語だけ抽出
        results[t[1]] = (words_only, len(words_only))
    return results

In [ ]:
#確認
results = longest_shiritori(word_list)

for r in results.items():
    print(r)

In [ ]:
def show_results(results):
  #出力を正規化して表示する
    for start, (path, length) in results.items():
        path_str = " → ".join(path)
        print(f"{start}: {path_str} (長さ={length})")

In [ ]:
#表示
show_results(results)

### 2.3 全体最長のしりとり結果

In [ ]:
def get_longest_all(results):
  #全体最長の結果のみを抽出してリストに格納
    max_len = 0
    longest_results = []

    for start, (path, length) in results.items():
        if length > max_len:
            max_len = length
            longest_results = [(start, path, length)]
        elif length == max_len:
            longest_results.append((start, path, length))

    return longest_results

In [ ]:
result_list = get_longest_all(results)
print(result_list)

In [ ]:
def show_longest_results(longest_results):
  #正規化して表示する関数
    for start, path, length in longest_results:
        path_str = " → ".join(path)
        print(f"{start}: {path_str} (長さ={length})")

In [ ]:
#正規化して表示
show_longest_results(result_list)

### 2.4 一括化

In [ ]:
def normalize(data):
  #正規化処理を関数化
  n_lists = []

  for token in t.tokenize(data):
    if token.part_of_speech.startswith("名詞"):
      if not (token.reading == "*"):
        n_lists.append((token.reading,token.surface))
      else:
        n_lists.append((jaconv.hira2kata(token.surface),token.surface))

  nr_list = remove_duplicates(n_lists)
  nrs_list = sorted(nr_list)

  return nrs_list

In [ ]:
def run_shiritori(data):
    #全処理の一括化
    tuple_list = normalize(data)
    processed = process_tuples(tuple_list)
    results = longest_shiritori(processed)
    result_list = get_longest_all(results)
    show_longest_results(result_list)

In [ ]:
#動作チェック
run_shiritori(data)

## 3. 様々な文章例

### 「檸檬」著：梶井基次郎

#### 全文

In [ ]:
lemon_all = """
えたいの知れない不吉な塊が私の心を始終圧おさえつけていた。焦躁と言おうか、嫌悪と言おうか――酒を飲んだあとに宿酔があるように、酒を毎日飲んでいると宿酔に相当した時期がやって来る。それが来たのだ。これはちょっといけなかった。結果した肺尖や神経衰弱がいけないのではない。また背を焼くような借金などがいけないのではない。いけないのはその不吉な塊だ。以前私を喜ばせたどんな美しい音楽も、どんな美しい詩の一節も辛抱がならなくなった。蓄音器を聴かせてもらいにわざわざ出かけて行っても、最初の二三小節で不意に立ち上がってしまいたくなる。何かが私を居堪たまらずさせるのだ。それで始終私は街から街を浮浪し続けていた。
何故だかその頃私は見すぼらしくて美しいものに強くひきつけられたのを覚えている。風景にしても壊れかかった街だとか、その街にしてもよそよそしい表通りよりもどこか親しみのある、汚い洗濯物が干してあったりがらくたが転がしてあったりむさくるしい部屋が覗いていたりする裏通りが好きであった。雨や風が蝕むしばんでやがて土に帰ってしまう、と言ったような趣きのある街で、土塀が崩れていたり家並が傾きかかっていたり――勢いのいいのは植物だけで、時とするとびっくりさせるような向日葵があったりカンナが咲いていたりする。時どき私はそんな路を歩きながら、ふと、そこが京都ではなくて京都から何百里も離れた仙台とか長崎とか――そのような市へ今自分が来ているのだ――という錯覚を起こそうと努める。私は、できることなら京都から逃げ出して誰一人知らないような市へ行ってしまいたかった。第一に安静。がらんとした旅館の一室。清浄な蒲団。匂においのいい蚊帳と糊のよくきいた浴衣。そこで一月ほど何も思わず横になりたい。希ねがわくはここがいつの間にかその市になっているのだったら。――錯覚がようやく成功しはじめると私はそれからそれへ想像の絵具を塗りつけてゆく。なんのことはない、私の錯覚と壊れかかった街との二重写しである。そして私はその中に現実の私自身を見失うのを楽しんだ。
私はまたあの花火というやつが好きになった。花火そのものは第二段として、あの安っぽい絵具で赤や紫や黄や青や、さまざまの縞模様を持った花火の束、中山寺の星下り、花合戦、枯れすすき。それから鼠花火というのは一つずつ輪になっていて箱に詰めてある。そんなものが変に私の心を唆そそった。
それからまた、びいどろという色硝子で鯛や花を打ち出してあるおはじきが好きになったし、南京玉が好きになった。またそれを嘗なめてみるのが私にとってなんともいえない享楽だったのだ。あのびいどろの味ほど幽かすかな涼しい味があるものか。私は幼い時よくそれを口に入れては父母に叱られたものだが、その幼時のあまい記憶が大きくなって落ちぶれた私に蘇よみがえってくる故だろうか、まったくあの味には幽かすかな爽さわやかななんとなく詩美と言ったような味覚が漂って来る。
察しはつくだろうが私にはまるで金がなかった。とは言えそんなものを見て少しでも心の動きかけた時の私自身を慰めるためには贅沢ということが必要であった。二銭や三銭のもの――と言って贅沢なもの。美しいもの――と言って無気力な私の触角にむしろ媚こびて来るもの。――そう言ったものが自然私を慰めるのだ。
生活がまだ蝕むしばまれていなかった以前私の好きであった所は、たとえば丸善であった。赤や黄のオードコロンやオードキニン。洒落た切子細工や典雅なロココ趣味の浮模様を持った琥珀色や翡翠色の香水壜。煙管、小刀、石鹸、煙草。私はそんなものを見るのに小一時間も費すことがあった。そして結局一等いい鉛筆を一本買うくらいの贅沢をするのだった。しかしここももうその頃の私にとっては重くるしい場所に過ぎなかった。書籍、学生、勘定台、これらはみな借金取りの亡霊のように私には見えるのだった。
ある朝――その頃私は甲の友達から乙の友達へというふうに友達の下宿を転々として暮らしていたのだが――友達が学校へ出てしまったあとの空虚な空気のなかにぽつねんと一人取り残された。私はまたそこから彷徨さまよい出なければならなかった。何かが私を追いたてる。そして街から街へ、先に言ったような裏通りを歩いたり、駄菓子屋の前で立ち留ったり、乾物屋の乾蝦や棒鱈や湯葉を眺めたり、とうとう私は二条の方へ寺町を下さがり、そこの果物屋で足を留めた。ここでちょっとその果物屋を紹介したいのだが、その果物屋は私の知っていた範囲で最も好きな店であった。そこは決して立派な店ではなかったのだが、果物屋固有の美しさが最も露骨に感ぜられた。果物はかなり勾配の急な台の上に並べてあって、その台というのも古びた黒い漆塗の板だったように思える。何か華やかな美しい音楽の快速調の流れが、見る人を石に化したというゴルゴンの鬼面――的なものを差しつけられて、あんな色彩やあんなヴォリウムに凝り固まったというふうに果物は並んでいる。青物もやはり奥へゆけばゆくほど堆高く積まれている。――実際あそこの人参葉の美しさなどは素晴らしかった。それから水に漬つけてある豆だとか慈姑だとか。
またそこの家の美しいのは夜だった。寺町通はいったいに賑やかな通りで――と言って感じは東京や大阪よりはずっと澄んでいるが――飾窓の光がおびただしく街路へ流れ出ている。それがどうしたわけかその店頭の周囲だけが妙に暗いのだ。もともと片方は暗い二条通に接している街角になっているので、暗いのは当然であったが、その隣家が寺町通にある家にもかかわらず暗かったのがはっきりしない。しかしその家が暗くなかったら、あんなにも私を誘惑するには至らなかったと思う。もう一つはその家の打ち出した廂なのだが、その廂が眼深に冠った帽子の廂のように――これは形容というよりも、「おや、あそこの店は帽子の廂をやけに下げているぞ」と思わせるほどなので、廂の上はこれも真暗なのだ。そう周囲が真暗なため、店頭に点つけられた幾つもの電燈が驟雨のように浴びせかける絢爛は、周囲の何者にも奪われることなく、ほしいままにも美しい眺めが照らし出されているのだ。裸の電燈が細長い螺旋棒を眼の中へ刺し込んでくる往来に立って、また近所にある鎰屋の二階の硝子窓をすかして眺めたこの果物店の眺めほど、その時どきの私を興がらせたものは寺町の中でも稀だった。
その日私はいつになくその店で買物をした。というのはその店には珍しい檸檬が出ていたのだ。檸檬などごくありふれている。がその店というのも見すぼらしくはないまでもただあたりまえの八百屋に過ぎなかったので、それまであまり見かけたことはなかった。いったい私はあの檸檬が好きだ。レモンの絵具をチューブから搾り出して固めたようなあの単純な色も、それからあの丈の詰まった紡錘形の恰好も。――結局私はそれを一つだけ買うことにした。
それからの私はどこへどう歩いたのだろう。私は長い間街を歩いていた。始終私の心を圧えつけていた不吉な塊がそれを握った瞬間からいくらか弛ゆるんで来たとみえて、私は街の上で非常に幸福であった。あんなに執拗だった憂鬱が、そんなものの一顆で紛らされる――あるいは不審なことが、逆説的なほんとうであった。それにしても心というやつはなんという不可思議なやつだろう。
その檸檬の冷たさはたとえようもなくよかった。その頃私は肺尖を悪くしていていつも身体に熱が出た。事実友達の誰彼に私の熱を見せびらかすために手の握り合いなどをしてみるのだが、私の掌が誰のよりも熱かった。その熱い故だったのだろう、握っている掌から身内に浸み透ってゆくようなその冷たさは快いものだった。
私は何度も何度もその果実を鼻に持っていっては嗅いでみた。それの産地だというカリフォルニアが想像に上って来る。漢文で習った「売柑者之言」の中に書いてあった「鼻を撲う」という言葉が断きれぎれに浮かんで来る。そしてふかぶかと胸一杯に匂やかな空気を吸い込めば、ついぞ胸一杯に呼吸したことのなかった私の身体や顔には温い血のほとぼりが昇って来てなんだか身内に元気が目覚めて来たのだった。
実際あんな単純な冷覚や触覚や嗅覚や視覚が、ずっと昔からこればかり探していたのだと言いたくなったほど私にしっくりしたなんて私は不思議に思える――それがあの頃のことなんだから。
私はもう往来を軽やかな昂奮に弾んで、一種誇りさえ感じながら、美的装束をして街を歩かした詩人のことなど思い浮かべては歩いていた。汚れた手拭の上へ載せてみたりマントの上へあてがってみたりして色の反映を量はかったり、またこんなことを思ったり、――つまりはこの重さなんだな。――その重さこそ常づね尋ねあぐんでいたもので、疑いもなくこの重さはすべての善いものすべての美しいものを重量に換算して来た重さであるとか、思いあがった諧謔心からそんな馬鹿げたことを考えてみたり――なにがさて私は幸福だったのだ。
どこをどう歩いたのだろう、私が最後に立ったのは丸善の前だった。平常あんなに避けていた丸善がその時の私にはやすやすと入れるように思えた。
「今日は一つ入ってみてやろう」そして私はずかずか入って行った。
しかしどうしたことだろう、私の心を充たしていた幸福な感情はだんだん逃げていった。香水の壜にも煙管にも私の心はのしかかってはゆかなかった。憂鬱が立て込めて来る、私は歩き廻った疲労が出て来たのだと思った。私は画本の棚の前へ行ってみた。画集の重たいのを取り出すのさえ常に増して力が要るな！と思った。しかし私は一冊ずつ抜き出してはみる、そして開けてはみるのだが、克明にはぐってゆく気持はさらに湧いて来ない。しかも呪われたことにはまた次の一冊を引き出して来る。それも同じことだ。それでいて一度バラバラとやってみなくては気が済まないのだ。それ以上は堪たまらなくなってそこへ置いてしまう。以前の位置へ戻すことさえできない。私は幾度もそれを繰り返した。とうとうおしまいには日頃から大好きだったアングルの橙色の重い本までなおいっそうの堪えがたさのために置いてしまった。――なんという呪われたことだ。手の筋肉に疲労が残っている。私は憂鬱になってしまって、自分が抜いたまま積み重ねた本の群を眺めていた。
以前にはあんなに私をひきつけた画本がどうしたことだろう。一枚一枚に眼を晒し終わって後、さてあまりに尋常な周囲を見廻すときのあの変にそぐわない気持を、私は以前には好んで味わっていたものであった。
「あ、そうだそうだ」その時私は袂の中の檸檬を憶い出した。本の色彩をゴチャゴチャに積みあげて、一度この檸檬で試してみたら。「そうだ」
私にまた先ほどの軽やかな昂奮が帰って来た。私は手当たり次第に積みあげ、また慌ただしく潰し、また慌しく築きあげた。新しく引き抜いてつけ加えたり、取り去ったりした。奇怪な幻想的な城が、そのたびに赤くなったり青くなったりした。
やっとそれはでき上がった。そして軽く跳りあがる心を制しながら、その城壁の頂きに恐る恐る檸檬を据えつけた。そしてそれは上出来だった。
見わたすと、その檸檬の色彩はガチャガチャした色の階調をひっそりと紡錘形の身体の中へ吸収してしまって、カーンと冴えかえっていた。私は埃っぽい丸善の中の空気が、その檸檬の周囲だけ変に緊張しているような気がした。私はしばらくそれを眺めていた。
不意に第二のアイディアが起こった。その奇妙なたくらみはむしろ私をぎょっとさせた。
――それをそのままにしておいて私は、なに喰わぬ顔をして外へ出る。――
私は変にくすぐったい気持がした。「出て行こうかなあ。そうだ出て行こう」そして私はすたすた出て行った。
変にくすぐったい気持が街の上の私を微笑ませた。丸善の棚へ黄金色に輝く恐ろしい爆弾を仕掛けて来た奇怪な悪漢が私で、もう十分後にはあの丸善が美術の棚を中心として大爆発をするのだったらどんなにおもしろいだろう。
私はこの想像を熱心に追求した。「そうしたらあの気詰まりな丸善も粉葉みじんだろう」
そして私は活動写真の看板画が奇体な趣きで街を彩っている京極を下って行った。
"""

#### 実プログラム1

In [ ]:
lemon = """
えたいの知れない不吉な塊が私の心を始終圧おさえつけていた。
焦躁と言おうか、嫌悪と言おうか――酒を飲んだあとに宿酔があるように、酒を毎日飲んでいると宿酔に相当した時期がやって来る。
それが来たのだ。これはちょっといけなかった。結果した肺尖や神経衰弱がいけないのではない。また背を焼くような借金などがいけないのではない。いけないのはその不吉な塊だ。

以前私を喜ばせたどんな美しい音楽も、どんな美しい詩の一節も辛抱がならなくなった。
蓄音器を聴かせてもらいにわざわざ出かけて行っても、最初の二三小節で不意に立ち上がってしまいたくなる。
何かが私を居堪たまらずさせるのだ。それで始終私は街から街を浮浪し続けていた。

何故だかその頃私は見すぼらしくて美しいものに強くひきつけられたのを覚えている。
風景にしても壊れかかった街だとか、その街にしてもよそよそしい表通りよりもどこか親しみのある、汚い洗濯物が干してあったりがらくたが転がしてあったりむさくるしい部屋が覗いていたりする裏通りが好きであった。
雨や風が蝕むしばんでやがて土に帰ってしまう、と言ったような趣きのある街で、土塀が崩れていたり家並が傾きかかっていたり――勢いのいいのは植物だけで、時とするとびっくりさせるような向日葵があったりカンナが咲いていたりする。
"""

In [ ]:
run_shiritori(lemon)

####実プログラム2

In [ ]:
#1003文字
lemon2 = """
えたいの知れない不吉な塊が私の心を始終圧おさえつけていた。焦躁と言おうか、嫌悪と言おうか――酒を飲んだあとに宿酔があるように、酒を毎日飲んでいると宿酔に相当した時期がやって来る。それが来たのだ。これはちょっといけなかった。結果した肺尖や神経衰弱がいけないのではない。また背を焼くような借金などがいけないのではない。いけないのはその不吉な塊だ。以前私を喜ばせたどんな美しい音楽も、どんな美しい詩の一節も辛抱がならなくなった。蓄音器を聴かせてもらいにわざわざ出かけて行っても、最初の二三小節で不意に立ち上がってしまいたくなる。何かが私を居堪たまらずさせるのだ。それで始終私は街から街を浮浪し続けていた。
何故だかその頃私は見すぼらしくて美しいものに強くひきつけられたのを覚えている。風景にしても壊れかかった街だとか、その街にしてもよそよそしい表通りよりもどこか親しみのある、汚い洗濯物が干してあったりがらくたが転がしてあったりむさくるしい部屋が覗いていたりする裏通りが好きであった。雨や風が蝕むしばんでやがて土に帰ってしまう、と言ったような趣きのある街で、土塀が崩れていたり家並が傾きかかっていたり――勢いのいいのは植物だけで、時とするとびっくりさせるような向日葵があったりカンナが咲いていたりする。時どき私はそんな路を歩きながら、ふと、そこが京都ではなくて京都から何百里も離れた仙台とか長崎とか――そのような市へ今自分が来ているのだ――という錯覚を起こそうと努める。私は、できることなら京都から逃げ出して誰一人知らないような市へ行ってしまいたかった。第一に安静。がらんとした旅館の一室。清浄な蒲団。匂においのいい蚊帳と糊のよくきいた浴衣。そこで一月ほど何も思わず横になりたい。希ねがわくはここがいつの間にかその市になっているのだったら。――錯覚がようやく成功しはじめると私はそれからそれへ想像の絵具を塗りつけてゆく。なんのことはない、私の錯覚と壊れかかった街との二重写しである。そして私はその中に現実の私自身を見失うのを楽しんだ。
私はまたあの花火というやつが好きになった。花火そのものは第二段として、あの安っぽい絵具で赤や紫や黄や青や、さまざまの縞模様を持った花火の束、中山寺の星下り、花合戦、枯れすすき。それから鼠花火というのは一つずつ輪になっていて箱に詰めてある。そんなものが変に私の心を唆そそった。
"""

In [ ]:
run_shiritori(lemon2)

### ニュース記事

In [ ]:
news = """
自公党首が連立巡り午後に再会談…献金規制強化焦点、自民の回答次第で協議打ち切りも

自民党の高市総裁と公明党の斉藤代表は１０日午後、国会内で改めて会談し、連立政権を継続するかどうかを協議する。公明は「政治とカネ」の問題に対する自民の姿勢に懸念を強めており、企業・団体献金の受け皿の大幅制限を自民が受け入れなければ、連立離脱もやむを得ないとの見方が強まっている。

　自公党首会談には、自民の鈴木、公明の西田両幹事長が同席する。自民内には「丁寧に議論を行う」（高市氏周辺）と協議継続に期待する向きもあるが、公明側は企業・団体献金の規制強化で納得できる回答がなければ協議を打ち切る構えを見せる。公明は９日夜の中央幹事会で、連立交渉の対応を斉藤、西田両氏に一任することを決めた。

　斉藤氏は９日夜に都内で開いた全国県代表協議会で、自民から十分な回答がない場合は、臨時国会で行われる首相指名選挙で「高市氏と書くわけにはいかない」と述べた。協議会の出席者からは連立維持を求める意見が出た一方で、「政治資金の問題は一歩も引かずに党の主張を貫くべきだ」などとして自民には譲歩せずに連立離脱を求める声も多く上がった。

　高市氏は９日夜のテレビ番組で、「自公連立は基本中の基本だ。政策合意の文書が早く作れるように頑張っていく」と語った。

　７日の党首会談では、斉藤氏が企業・団体献金の受け取りについて都道府県組織などに大幅に絞る案を示したが、折り合いがつかなかった。自民は「地方議員が献金を受け取れなくなれば、活動に支障が出る」と反論する。

　高市氏は９日のテレビ番組で「県連に全部お金が集まるとチェックが働かないのでは」と述べ、透明性の低下を招く可能性があると指摘した。
"""

In [ ]:
run_shiritori(news)